In [1]:
import math # math functions
import numpy as np
import cv2
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential, Model
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Masking, Dropout, Conv1D, Conv2D, MaxPooling2D, TimeDistributed, Flatten, MaxPool1D
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
batch_size=16

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
        'screenshot_dataset\\train',
        target_size=(172, 310),
        batch_size=batch_size,
        class_mode='binary')

val_generator = val_datagen.flow_from_directory(
        'screenshot_dataset\\val',
        target_size=(172, 310),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        'screenshot_dataset\\test',
        target_size=(172, 310),
        batch_size=batch_size,
        class_mode='binary')

Found 652 images belonging to 2 classes.
Found 164 images belonging to 2 classes.
Found 206 images belonging to 2 classes.


In [5]:
input1 = Input(shape=(172,310,3))

x = Conv2D(256, (3, 3), activation="relu", padding="same")(input1)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(256, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(256, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(256, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)
x = Dense(256, activation="relu")(x)

x = Dropout(0.2)(x)
out = Dense(1, activation="sigmoid")(x)

model = Model(inputs=input1, outputs=out)
opt=Adam(lr=0.01,decay=0.00001)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 172, 310, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 172, 310, 256)     7168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 86, 155, 256)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 86, 155, 256)      590080    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 43, 77, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 43, 77, 256)       590080    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 21, 38, 256)       0         
__________

In [7]:
model.fit_generator(
    generator=train_generator,
    steps_per_epoch=train_generator.n//train_generator.batch_size,
    epochs=30,
    verbose=1,
    validation_data=val_generator,
    validation_steps=val_generator.n//val_generator.batch_size,
)

Epoch 1/30


ResourceExhaustedError: OOM when allocating tensor with shape[204288,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/Adam/mul_34 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/Adam/sub_14, training/Adam/gradients/dense_1/MatMul_grad/MatMul_1)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
pred=model.predict_generator(test_generator,verbose=1)

In [ ]:
pred

In [ ]:
pred2=model.predict_generator(train_generator,verbose=1)

In [ ]:
pred2